<p style="font-weight:bold;"> <span style="font-size: 36px"> Scenario Transactional Data Import Test </span> </p>

In [0]:
#!eval-notebook "../Initialization/InitSystemorphBaseToMemory"

In [0]:
DataSource.Reset(x => x.ResetType<RawVariable>().ResetType<IfrsVariable>());

# Setting Constants

In [0]:
var EnableScenarioBackup = Scenarios.EnableScenario;

In [0]:
Scenarios.EnableScenario = true;

In [0]:
var argsBestEstimate = new ImportArgs ("CH", 2020, 12, Periodicity.Quarterly, null, ImportFormats.Cashflow);
var argsScenario = new ImportArgs ("CH", 2020, 12, Periodicity.Quarterly, "MTUP10pct", ImportFormats.Cashflow);

In [0]:
var estimateTypes = (await DataSource.Query<EstimateType>().ToArrayAsync());
var actualEstimateTypes = estimateTypes.Where(et => et.InputSource.Contains(InputSource.Actual)).Select(x => x.SystemName);
var cashflowEstimateTypes = estimateTypes.Where(et => et.InputSource.Contains(InputSource.Cashflow)).Select(x => x.SystemName);
var actualAndCashflowEstimateTypes = actualEstimateTypes.Intersect(cashflowEstimateTypes);
var onlyActualEstimateTypes = actualEstimateTypes.Except(cashflowEstimateTypes);
var onlyCashflowEstimateTypes = cashflowEstimateTypes.Except(actualEstimateTypes);

# Best Estimate Initialization

In [0]:
var cashflowBestEstimateInit = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,
@@Cashflow
DataNode,AmountType,EstimateType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,PR,BE,CL,C,,100,0,0,100,0,0,100,0,0,100,0,0,0,100,0,0,100,0,0,100,0,0,100,0
DT1.1,NIC,BE,CL,C,,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25
DT1.1,,CU,CL,C,,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-3
DT1.1,,RA,CL,C,,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5
DTR1.1,PR,BE,CL,C,,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0
DTR1.1,NIC,BE,CL,C,,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5
DTR1.1,,CU,CL,C,,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
DTR1.1,,RA,CL,C,,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25";

In [0]:
await Import.FromString(cashflowBestEstimateInit).WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

In [0]:
var actualsBestEstimateInit = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,
@@Actual
DataNode,AocType,AmountType,EstimateType,AccidentYear,Value
DT1.1,CF,ACA,A,,-10
DT1.1,CF,AEA,A,,-5
DT1.1,CF,NIC,A,,-280
DT1.1,CF,PR,A,,400
DTR1.1,CF,PR,A,,-200
DTR1.1,CF,NIC,A,,140";

In [0]:
await Import.FromString(actualsBestEstimateInit).WithFormat(ImportFormats.Actual).WithTarget(DataSource).ExecuteAsync()

In [0]:
var ivsBestEstimateInit = await DataSource.Query<IfrsVariable>().ToArrayAsync();

# Scenario Init Cash flows

In [0]:
var cashflowsScenario = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,MTUP10pct
@@Cashflow
DataNode,AmountType,EstimateType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,PR,BE,CL,C,,110,0,0,110,0,0,110,0,0,110,0,0,0,110,0,0,110,0,0,110,0,0,110,0
DT1.1,NIC,BE,CL,C,,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5
DT1.1,,CU,CL,C,,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-53.5,-52.5,-31.3
DT1.1,,RA,CL,C,,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75";

In [0]:
await Import.FromString(cashflowsScenario).WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenario);
var ivsScenarioInitCashFlows = await DataSource.Query<IfrsVariable>().ToArrayAsync();
var diffs = ivsScenarioInitCashFlows.Except(ivsBestEstimateInit, IfrsVariableComparer.Instance());
var actualVars = diffs.Where(x => onlyActualEstimateTypes.Contains(x.EstimateType)).ToArray();
var cashflowVars = diffs.Where(x => onlyCashflowEstimateTypes.Contains(x.EstimateType)).ToArray();
var actualAndCashflowVars = diffs.Where(x => actualAndCashflowEstimateTypes.Contains(x.EstimateType)).ToArray();

In [0]:
actualVars.Length.Should().Be(0);
cashflowVars.Length.Should().NotBe(0);
diffs.Except(actualVars.Union(cashflowVars).Union(actualAndCashflowVars)).ToArray().Length.Should().Be(0);

# Scenario Init Actuals

In [0]:
var actualsScenario = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,MTUP10pct
@@Actual
DataNode,AocType,AmountType,EstimateType,AccidentYear,Value
DT1.1,CF,NIC,A,,-308";

In [0]:
await Import.FromString(actualsScenario).WithFormat(ImportFormats.Actual).WithTarget(DataSource).ExecuteAsync()

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenario);
var ivsScenarioInitActuals = DataSource.Query<IfrsVariable>().ToArray();
var diffs = ivsScenarioInitActuals.Except(ivsScenarioInitCashFlows, IfrsVariableComparer.Instance());
var actualVars = diffs.Where(x => onlyActualEstimateTypes.Contains(x.EstimateType)).ToArray();
var cashflowVars = diffs.Where(x => onlyCashflowEstimateTypes.Contains(x.EstimateType)).ToArray();
var actualAndCashflowVars = diffs.Where(x => actualAndCashflowEstimateTypes.Contains(x.EstimateType)).ToArray();

In [0]:
actualVars.Length.Should().NotBe(0);
cashflowVars.Length.Should().Be(0);
diffs.Except(actualVars.Union(cashflowVars).Union(actualAndCashflowVars)).ToArray().Length.Should().Be(0);

# Update of Deferrable Actuals with a Scenario Coverage Units Import

Update CU cashflow for scenario and check that the Amortization Factor (and hence the Deferrable Actual) is updated

In [0]:
var cashflowsScenarioUpdateCU = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,MTUP10pct
@@Cashflow
DataNode,AmountType,EstimateType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,,CU,CL,C,,-17,-17,-17,-17,-17,-17,-17,-17,-17,-17,-17,-17,-17,-6.5,-6.5,-6.5,-6.5,-6.5,-6.5,-6.5,-6.5,-6.5,-7.5,-8.5";

In [0]:
await Import.FromString(cashflowsScenarioUpdateCU).WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenario);
var ivsScenarioCashflowUpdateCU = await DataSource.Query<IfrsVariable>().ToArrayAsync();
var diffs = ivsScenarioCashflowUpdateCU.Except(ivsScenarioInitActuals, IfrsVariableComparer.Instance());
var actualVars = diffs.Where(x => onlyActualEstimateTypes.Contains(x.EstimateType)).ToArray();
var cashflowVars = diffs.Where(x => onlyCashflowEstimateTypes.Contains(x.EstimateType)).ToArray();
var actualAndCashflowVars = diffs.Where(x => actualAndCashflowEstimateTypes.Contains(x.EstimateType)).ToArray();

In [0]:
actualVars.Length.Should().Be(0);
cashflowVars.Where(x => x.EstimateType == EstimateTypes.F).Count().Should().NotBe(0);
diffs.Where(x => x.EstimateType == EstimateTypes.C).Count().Should().NotBe(0);
diffs.Where(x => x.EstimateType == EstimateTypes.DA).Count().Should().NotBe(0);
diffs.Except(actualVars.Union(cashflowVars).Union(actualAndCashflowVars)).ToArray().Length.Should().Be(0);

# Update of Deferrable Actuals with a Scenario ACA/AEA Import

In [0]:
var actualsScenarioUpdateACAAEA = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,MTUP10pct
@@Actual
DataNode,AocType,AmountType,EstimateType,AccidentYear,Value
DT1.1,CF,ACA,A,,-555
DT1.1,CF,AEA,A,,-777";

In [0]:
await Import.FromString(actualsScenarioUpdateACAAEA).WithFormat(ImportFormats.Actual).WithTarget(DataSource).ExecuteAsync()

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenario);
var ivsScenarioActualUpdateACAAEA = await DataSource.Query<IfrsVariable>().ToArrayAsync();
var diffs = ivsScenarioActualUpdateACAAEA.Except(ivsScenarioCashflowUpdateCU, IfrsVariableComparer.Instance());
var actualVars = diffs.Where(x => onlyActualEstimateTypes.Contains(x.EstimateType)).ToArray();
var cashflowVars = diffs.Where(x => onlyCashflowEstimateTypes.Contains(x.EstimateType)).ToArray();
var actualAndCashflowVars = diffs.Where(x => actualAndCashflowEstimateTypes.Contains(x.EstimateType)).ToArray();

In [0]:
cashflowVars.Count().Should().Be(0);
diffs.Where(x => x.EstimateType == EstimateTypes.DA).ToArray().Length.Should().NotBe(0);
diffs.Where(x => x.EstimateType == EstimateTypes.F).ToArray().Length.Should().Be(0);
diffs.Except(actualVars.Union(cashflowVars).Union(actualAndCashflowVars)).ToArray().Length.Should().Be(0);

# Removal of Deferrable Actuals with a Actual Scenario Import

Set ACA and AEA to zero and check that DA is removed

In [0]:
var actualsScenarioNoACAAEA = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,MTUP10pct
@@Actual
DataNode,AocType,AmountType,EstimateType,AccidentYear,Value
DT1.1,CF,ACA,A,,0
DT1.1,CF,AEA,A,,0";

In [0]:
await Import.FromString(actualsScenarioNoACAAEA).WithFormat(ImportFormats.Actual).WithTarget(DataSource).ExecuteAsync()

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenario);
var ivsScenarioActualNoACAAEA = await DataSource.Query<IfrsVariable>().ToArrayAsync();
var diffs = ivsScenarioActualNoACAAEA.Except(ivsScenarioActualUpdateACAAEA, IfrsVariableComparer.Instance());

In [0]:
diffs.Where(x => x.EstimateType == EstimateTypes.DA).ToArray().Length.Should().NotBe(0);
ivsScenarioActualNoACAAEA.Where(x => x.EstimateType == EstimateTypes.F).Count().Should().NotBe(0);
ivsScenarioActualNoACAAEA.Where(x => x.EstimateType == EstimateTypes.DA && x.Values.Any(v => Math.Abs(v) > Precision)).Count().Should().Be(0);

# Robustness: scenario back to Best Estimate

In [0]:
var cashflowsScenarioEqualToBestEstimate = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,MTUP10pct
@@Cashflow
DataNode,AmountType,EstimateType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,PR,BE,CL,C,,100,0,0,100,0,0,100,0,0,100,0,0,0,100,0,0,100,0,0,100,0,0,100,0
DT1.1,NIC,BE,CL,C,,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25
DT1.1,,CU,CL,C,,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-3
DT1.1,,RA,CL,C,,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5";

In [0]:
var actualsScenarioEqualToBestEstimate = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,MTUP10pct
@@Actual
DataNode,AocType,AmountType,EstimateType,AccidentYear,Value
DT1.1,CF,ACA,A,,-10
DT1.1,CF,AEA,A,,-5
DT1.1,CF,NIC,A,,-280";

In [0]:
await Import.FromString(actualsScenarioEqualToBestEstimate).WithFormat(ImportFormats.Actual).WithTarget(DataSource).ExecuteAsync()

In [0]:
await Import.FromString(cashflowsScenarioEqualToBestEstimate).WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenario);
var ivsScenarioReimportEqualToBestEstimate = await DataSource.Query<IfrsVariable>().ToArrayAsync(); 
var diffs = ivsScenarioReimportEqualToBestEstimate.Except(ivsBestEstimateInit, IfrsVariableComparer.Instance()).Where(x =>x.Values.Any(v => Math.Abs(v) > Precision));

In [0]:
ivsScenarioReimportEqualToBestEstimate.Count().Should().Be(0);

# Reimport of Best Estimate Cashflow: zero impact on Scenario

Reimport of best estimate with different gross cash flow, whereas reinsurance business is the same of the initialization. This causes an update of the best estimate IFRS variables but it must have no impact on Scenario variables because gross cash flow contains only a combination of AocType, Novelty and AmoutType that have already been imported for Scenario (see above the variable cashflowsScenarioEqualToBestEstimate). 

In [0]:
var cashflowsBestEstimateNoImpact = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,
@@Cashflow
DataNode,AmountType,EstimateType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,PR,BE,CL,C,,210,0,0,210,0,0,210,0,0,210,0,0,0,210,0,0,210,0,0,210,0,0,210,0
DT1.1,NIC,BE,CL,C,,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37
DT1.1,,CU,CL,C,,-21,-21,-21,-21,-21,-21,-21,-21,-21,-21,-21,-21,-21,-17,-17,-17,-17,-17,-17,-17,-17,-17,-17,-3.3
DT1.1,,RA,CL,C,,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42
DTR1.1,PR,BE,CL,C,,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0
DTR1.1,NIC,BE,CL,C,,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5
DTR1.1,,CU,CL,C,,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
DTR1.1,,RA,CL,C,,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25";

In [0]:
await Import.FromString(cashflowsBestEstimateNoImpact).WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsBestEstimate);
var ivsBestEstimateReimportCashFlowsNoImpactScenario = await DataSource.Query<IfrsVariable>().ToArrayAsync();
var diffs = ivsBestEstimateReimportCashFlowsNoImpactScenario.Except(ivsBestEstimateInit, IfrsVariableComparer.Instance());
var actualVars = diffs.Where(x => onlyActualEstimateTypes.Contains(x.EstimateType)).ToArray();
var cashflowVars = diffs.Where(x => onlyCashflowEstimateTypes.Contains(x.EstimateType)).ToArray();
var actualAndCashflowVars = diffs.Where(x => actualAndCashflowEstimateTypes.Contains(x.EstimateType)).ToArray();

In [0]:
diffs.Where(x => x.EstimateType == EstimateTypes.DA).Count().Should().NotBe(0);
actualVars.Length.Should().Be(0);
diffs.Except(actualVars.Union(cashflowVars).Union(actualAndCashflowVars)).ToArray().Length.Should().Be(0);

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenario);
var ivsScenarioReimportCashFlowsNoImpactScenario = await DataSource.Query<IfrsVariable>().ToArrayAsync();
var diffs = ivsScenarioReimportCashFlowsNoImpactScenario.Except(ivsBestEstimateInit, IfrsVariableComparer.Instance());

In [0]:
diffs.Where(x => onlyCashflowEstimateTypes.Contains(x.EstimateType) && x.EstimateType != EstimateTypes.F && x.Values.Any(v => Math.Abs(v) > Precision)).Count().Should().Be(0);
diffs.Where(x => onlyActualEstimateTypes.Contains(x.EstimateType) && x.Values.Any(v => Math.Abs(v) > Precision)).Count().Should().Be(0);

# Reimport of Best Estimate Cashflow

In [0]:
var cashflowsBestEstimateImpact = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,
@@Cashflow
DataNode,AmountType,EstimateType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,ICO,BE,CL,C,,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3
DT1.1,PR,BE,CL,C,,210,0,0,210,0,0,210,0,0,210,0,0,0,210,0,0,210,0,0,210,0,0,210,0
DT1.1,NIC,BE,CL,C,,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37
DT1.1,,CU,CL,C,,-21,-21,-21,-21,-21,-21,-21,-21,-21,-21,-21,-21,-21,-17,-17,-17,-17,-17,-17,-17,-17,-17,-17,-3.3
DT1.1,,RA,CL,C,,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42";

In [0]:
await Import.FromString(cashflowsBestEstimateImpact).WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsBestEstimate);
var ivsBestEstimateReimportCashFlows = await DataSource.Query<IfrsVariable>().ToArrayAsync();
var diffs = ivsBestEstimateReimportCashFlows.Except(ivsBestEstimateReimportCashFlowsNoImpactScenario, IfrsVariableComparer.Instance());

In [0]:
diffs.Count().Should().NotBe(0);

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenario);
var ivsScenarioReimportCashFlows = await DataSource.Query<IfrsVariable>().ToArrayAsync();
var diffs = ivsScenarioReimportCashFlows.Except(ivsScenarioReimportCashFlowsNoImpactScenario, IfrsVariableComparer.Instance());
var actualVars = diffs.Where(x => onlyActualEstimateTypes.Contains(x.EstimateType)).ToArray();
var cashflowVars = diffs.Where(x => onlyCashflowEstimateTypes.Contains(x.EstimateType)).ToArray();
var actualAndCashflowVars = diffs.Where(x => actualAndCashflowEstimateTypes.Contains(x.EstimateType)).ToArray();

In [0]:
cashflowVars.Length.Should().NotBe(0);
actualVars.Length.Should().Be(0);
diffs.Except(actualVars.Union(cashflowVars).Union(actualAndCashflowVars)).ToArray().Length.Should().Be(0);

# Reimport of Best Estimate Actuals

In [0]:
var newActualsBestEstimate = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,
@@Actual
DataNode,AocType,AmountType,EstimateType,AccidentYear,Value
DT1.1,CF,NIC,A,,-2000
DT1.1,CF,PR,A,,4000
DT1.1,CF,CL,A,,-3000";

In [0]:
await Import.FromString(newActualsBestEstimate).WithFormat(ImportFormats.Actual).WithTarget(DataSource).ExecuteAsync()

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsBestEstimate);
var ivsBestEstimateReimportActuals = await DataSource.Query<IfrsVariable>().ToArrayAsync();
var diffs = ivsBestEstimateReimportActuals.Except(ivsBestEstimateReimportCashFlows, IfrsVariableComparer.Instance());
var actualVars = diffs.Where(x => onlyActualEstimateTypes.Contains(x.EstimateType)).ToArray();
var cashflowVars = diffs.Where(x => onlyCashflowEstimateTypes.Contains(x.EstimateType)).ToArray();

In [0]:
actualVars.Length.Should().NotBe(0);
cashflowVars.Length.Should().Be(0);

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenario);
var ivsScenarioReimportActuals = await DataSource.Query<IfrsVariable>().ToArrayAsync();
var diffs = ivsScenarioReimportActuals.Except(ivsScenarioReimportCashFlows, IfrsVariableComparer.Instance());
var actualVars = diffs.Where(x => onlyActualEstimateTypes.Contains(x.EstimateType)).ToArray();
var cashflowVars = diffs.Where(x => onlyCashflowEstimateTypes.Contains(x.EstimateType)).ToArray();

In [0]:
diffs.Where(x => x.EstimateType == EstimateTypes.C).Count().Should().NotBe(0);
actualVars.Length.Should().Be(0);
cashflowVars.Length.Should().Be(0);

# Update of Deferrable Actuals with a Best Estimate Coverage Units Import

Reset Coverage Units for Scenario so that they are taken from Best Estimate. Then, test that a re-import of Best Estimate Coverage Units changes the Scenario Deferrable Actuals. 

In [0]:
var cashflowsScenarioNoCU = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,MTUP10pct
@@Cashflow
DataNode,AmountType,EstimateType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,PR,BE,CL,C,,110,0,0,110,0,0,110,0,0,110,0,0,0,110,0,0,110,0,0,110,0,0,110,0
DT1.1,NIC,BE,CL,C,,-274.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5
DT1.1,,RA,CL,C,,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75";

In [0]:
var cashflowsBestEstimateUpdatedCU = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,
@@Cashflow
DataNode,AmountType,EstimateType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,PR,BE,CL,C,,200,0,0,200,0,0,200,0,0,200,0,0,0,200,0,0,200,0,0,200,0,0,200,0
DT1.1,NIC,BE,CL,C,,-33,-33,-33,-33,-33,-33,-33,-33,-33,-33,-33,-33,-33,-33,-33,-33,-33,-33,-33,-33,-33,-33,-33,-33
DT1.1,,CU,CL,C,,-719,-71,-70,-70,-69,-69,-68,-68,-67,-67,-66,-66,-65,-65,-64,-64,-63,-63,-62,-62,-61,-61,-60,-60
DT1.1,,RA,CL,C,,-45,-45,-45,-45,-45,-45,-45,-45,-45,-45,-45,-45,-45,-45,-45,-45,-45,-45,-45,-45,-45,-45,-45,-45";

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenario);
await DataSource.DeleteAsync(await DataSource.Query<RawVariable>().ToArrayAsync());

In [0]:
await Import.FromString(cashflowsScenarioNoCU).WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenario);
var ivsScenarioReimportCashflowNoCU = await DataSource.Query<IfrsVariable>().ToArrayAsync();

In [0]:
await Import.FromString(cashflowsBestEstimateUpdatedCU).WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenario);
var ivsScenarioReimportCashFlowsWithCU = await DataSource.Query<IfrsVariable>().ToArrayAsync();
var diffs = ivsScenarioReimportCashFlowsWithCU.Except(ivsScenarioReimportCashflowNoCU, IfrsVariableComparer.Instance());
var actualVars = diffs.Where(x => onlyActualEstimateTypes.Contains(x.EstimateType)).ToArray();
var cashflowVars = diffs.Where(x => onlyCashflowEstimateTypes.Contains(x.EstimateType)).ToArray();

In [0]:
actualVars.Length.Should().Be(0);
diffs.Count().Should().NotBe(0);

# Restore Constants

In [0]:
Scenarios.EnableScenario = EnableScenarioBackup;